In [1]:
import pandas as pd
import duckdb

from common.loader import *

In [2]:
products = get_bronze_dataset(BronzeDataName.PRODUCTS)
category_eng = get_bronze_dataset(BronzeDataName.CATEGORY)

(32951, 9)
(71, 2)


In [3]:
products.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


In [4]:
category_eng.head()

,product_category_name,product_category_name_english
0,beleza_saude,health_beauty
1,informatica_acessorios,computers_accessories
2,automotivo,auto
3,cama_mesa_banho,bed_bath_table
4,moveis_decoracao,furniture_decor


In [5]:
category_origin = duckdb.query("select distinct product_category_name from products where product_category_name is not null")
category_origin

┌────────────────────────────────────────────────┐
│             product_category_name              │
│                    varchar                     │
├────────────────────────────────────────────────┤
│ cama_mesa_banho                                │
│ moveis_escritorio                              │
│ cine_foto                                      │
│ artes                                          │
│ utilidades_domesticas                          │
│ brinquedos                                     │
│ casa_construcao                                │
│ moveis_cozinha_area_de_servico_jantar_e_jardim │
│ fashion_roupa_feminina                         │
│ casa_conforto_2                                │
│     ·                                          │
│     ·                                          │
│     ·                                          │
│ cool_stuff                                     │
│ eletrodomesticos                               │
│ fashion_bolsas_e_acessorios  

In [6]:
duckdb.query("select count(*) from category_eng")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│           71 │
└──────────────┘

In [7]:
duckdb.query("select * from category_eng limit 0")

┌───────────────────────┬───────────────────────────────┐
│ product_category_name │ product_category_name_english │
│        varchar        │            varchar            │
├───────────────────────┴───────────────────────────────┤
│                        0 rows                         │
└───────────────────────────────────────────────────────┘

In [8]:
category_origin_eng = duckdb.query("select category_origin.product_category_name as origin, category_eng.product_category_name_english as eng from category_origin left join category_eng on category_origin.product_category_name = category_eng.product_category_name")
category_origin_eng

┌───────────────────────────────┬────────────────────────────────┐
│            origin             │              eng               │
│            varchar            │            varchar             │
├───────────────────────────────┼────────────────────────────────┤
│ cama_mesa_banho               │ bed_bath_table                 │
│ moveis_escritorio             │ office_furniture               │
│ cine_foto                     │ cine_photo                     │
│ bebes                         │ baby                           │
│ instrumentos_musicais         │ musical_instruments            │
│ papelaria                     │ stationery                     │
│ alimentos_bebidas             │ food_drink                     │
│ flores                        │ flowers                        │
│ perfumaria                    │ perfumery                      │
│ pc_gamer                      │ NULL                           │
│    ·                          │  ·                          

In [9]:
duckdb.query("select * from category_origin_eng where eng is null")
# gaming_pc, portable_kitchen_and_food_preparators

┌───────────────────────────────────────────────┬─────────┐
│                    origin                     │   eng   │
│                    varchar                    │ varchar │
├───────────────────────────────────────────────┼─────────┤
│ portateis_cozinha_e_preparadores_de_alimentos │ NULL    │
│ pc_gamer                                      │ NULL    │
└───────────────────────────────────────────────┴─────────┘

In [10]:
# duckdb.register('category_origin_eng', category_origin_eng)
# duckdb.query("update category_origin_eng set eng='gaming_pc' where origin='pc_gamer'")
category_origin_eng_df = category_origin_eng.to_df()
category_origin_eng_df[category_origin_eng_df['origin'] == 'pc_gamer'] = 'gaming_pc'
category_origin_eng_df[category_origin_eng_df['origin'] == 'portateis_cozinha_e_preparadores_de_alimentos'] = 'portable_kitchen_and_food_preparators'

category_origin_eng_df.isnull().count()

origin    73
eng       73
dtype: int64

In [11]:
save_path = os.path.join(SILVER_DIR, "product_categories.csv")
category_origin_eng_df.to_csv(save_path, index=False)

In [12]:
category_origin_eng_df.head()

,origin,eng
0,artes,art
1,utilidades_domesticas,housewares
2,brinquedos,toys
3,casa_construcao,home_construction
4,moveis_cozinha_area_de_servico_jantar_e_jardim,kitchen_dining_laundry_garden_furniture


In [13]:
category_origin2eng = pd.merge(products.product_category_name, category_origin_eng_df, left_on='product_category_name', right_on='origin', how='left')
products.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


In [14]:
products.product_category_name = category_origin2eng['eng']
products.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumery,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,art,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,sports_leisure,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,baby,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,housewares,37.0,402.0,4.0,625.0,20.0,17.0,13.0


In [15]:
save_path = os.path.join(SILVER_DIR, "products.csv")
products.to_csv(save_path,index=False)